<a href="https://colab.research.google.com/github/eliasantoro99/AI-REPO/blob/main/training_Vision_T.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# STEP 1: INSTALLAZIONE LIBRERIE (da terminale)
# pip install roboflow torch torchvision timm tqdm matplotlib

# === IMPORT DELLE LIBRERIE NECESSARIE ===
from roboflow import Roboflow
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm, trange
import timm                         # Modelli pre-addestrati (come Vision Transformer)
import os                           # Gestione file e directory
import numpy as np

# === STEP 2: SCARICO IL DATASET DA ROBOFLOW ===
print("⬇️ Scarico il dataset da Roboflow...")

# Inizializzo Roboflow con la tua chiave API
rf = Roboflow(api_key="GAQ2cbOSPLavcNiilrPr")

# Workspace e progetto specifico
project = rf.workspace("roboanime").project("roboanimeproject")

# Scarico la versione 2 del dataset nella cartella "folder"
dataset = project.version(2).download("folder")
data_dir = dataset.location

print(f"✅ Dataset salvato in: {data_dir}")

# === STEP 3: PARAMETRI DI TRAINING ===
batch_size = 32            # Immagini per batch
num_epochs = 20            # Più epoche per apprendimento profondo
lr = 0.001                 # Learning rate (velocità di apprendimento)
patience = 3               # Early stopping: numero max epoche senza miglioramento

# Conta quante classi ci sono (cartelle nella directory train)
num_classes = len(os.listdir(os.path.join(data_dir, "train")))

# Device: preferisci GPU se disponibile
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"📊 Numero classi: {num_classes}")
print(f"🖥️  Training su: {device}")

# === STEP 4: TRASFORMAZIONI IMMAGINI ===
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ViT accetta immagini 224x224
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],  # Media RGB per ImageNet
                         [0.229, 0.224, 0.225])  # Std dev RGB per ImageNet
])

# === STEP 5: CARICAMENTO DEI DATI ===
print("📂 Carico immagini...")

train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, "valid"), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

print(f"✅ Train: {len(train_dataset)} immagini | Valid: {len(val_dataset)} immagini")

# === STEP 6: MODELLO VISION TRANSFORMER ===
print("🧠 Creo modello ViT pre-addestrato...")

model = timm.create_model('vit_base_patch16_224', pretrained=True)
model.head = nn.Linear(model.head.in_features, num_classes)
model = model.to(device)

# === STEP 7: FUNZIONE DI PERDITA E OTTIMIZZATORE ===
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# === STEP 8: CICLO DI TRAINING + EARLY STOPPING ===
print("🚀 Inizio training...")

best_val_loss = np.inf   # Inizializza la miglior validazione vista
epochs_no_improve = 0    # Contatore per early stopping

for epoch in trange(num_epochs, desc="Epoche"):
    model.train()
    train_loss = 0.0
    correct = 0
    total = 0

    loop = tqdm(train_loader, desc=f"Train Epoca {epoch+1}", leave=False)
    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = outputs.max(1)
        correct += preds.eq(labels).sum().item()
        total += labels.size(0)

        loop.set_postfix(loss=loss.item(), acc=f"{100*correct/total:.2f}%")

    avg_train_loss = train_loss / len(train_loader)
    train_acc = 100 * correct / total
    print(f"[Epoch {epoch+1}] 📉 Loss train: {avg_train_loss:.4f} | 🎯 Acc train: {train_acc:.2f}%")

    # === VALIDAZIONE DOPO OGNI EPOCA ===
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = outputs.max(1)
            val_correct += preds.eq(labels).sum().item()
            val_total += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_acc = 100 * val_correct / val_total
    print(f"🔍 Validazione — Loss: {avg_val_loss:.4f} | 🎯 Acc: {val_acc:.2f}%")

    # EARLY STOPPING: controllo miglioramento
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), "best_vit_model.pth")  # Salva miglior modello
        print("💾 Nuovo modello salvato!")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("🛑 Early stopping attivato: nessun miglioramento.")
            break

# === STEP 9: ACCURACY FINALE ===
print(f"\n✅ Accuracy finale validazione: {val_acc:.2f}%")


⬇️ Scarico il dataset da Roboflow...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to RoboAnimeProject-2 in folder:: 100%|██████████| 1023/1023 [00:00<00:00, 10555.98it/s]


✅ Dataset salvato in: /content/RoboAnimeProject-2
📊 Numero classi: 6
🖥️  Training su: cuda
📂 Carico immagini...
✅ Train: 700 immagini | Valid: 200 immagini
🧠 Creo modello ViT pre-addestrato...



The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

...

[Epoch 6] 📉 Loss train: 1.4409 | 🎯 Acc train: 37.71%
🔍 Validazione — Loss: 1.7398 | 🎯 Acc: 37.50%


Train Epoca 7:   0%|          | 0/22 [00:00<?, ?it/s]

[Epoch 7] 📉 Loss train: 1.4707 | 🎯 Acc train: 39.71%
🔍 Validazione — Loss: 1.7790 | 🎯 Acc: 41.50%


Train Epoca 8:   0%|          | 0/22 [00:00<?, ?it/s]

[Epoch 8] 📉 Loss train: 1.4579 | 🎯 Acc train: 38.43%
🔍 Validazione — Loss: 1.8523 | 🎯 Acc: 41.50%
🛑 Early stopping attivato: nessun miglioramento.

✅ Accuracy finale validazione: 41.50%
